In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

## Basic Agent

In [7]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions= "You are a helpful assistant. Respond only in caps.",
    model = 'gpt-4o-mini'
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

"HELLO! I'M JUST A PROGRAM, BUT I'M READY TO HELP YOU! HOW CAN I ASSIST YOU TODAY?"

## Joke Agent

In [11]:
joke_agent = Agent(
    name = "Joke Agent",
    instructions= "You are a joke teller. You are given a topic and you need to tell a joke about it."
)

topic = "History"
result = await Runner.run(joke_agent, topic)
result.final_output

'Why was the history teacher always so calm?\n\nBecause she had a lot of dates to fall back on!'

## Language Agent

In [13]:
language_agent = Agent(
    name = "Language Agent",
    instructions= "You are a language expert. You are given a joke and you need to rewrite it in a different language."
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Italy: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the historian break up with the librarian?

Too much drama—he wanted more of a non-friction relationship!

Translated joke:
Perché lo storico ha lasciato il bibliotecario?

Troppi drammi—voleva una relazione più "non-fiction"!


## Structured Outputs

### Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [17]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title : str
    ingredients : list[str]
    cooking_time : int #in minutes
    servings : int

recipe_agent = Agent(
    name= "Recipe Agent",
    instructions= ("You are an agent for creating recipes. You will be given the name of a food and your job"
                   "is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type = Recipe
)

response = await Runner.run(recipe_agent, "Italian sausage with spaghetti")
recipe = response.final_output
print(recipe)

title='Italian Sausage with Spaghetti' ingredients=['8 oz spaghetti', '4 Italian sausages', '2 tablespoons olive oil', '1 onion, finely chopped', '2 cloves garlic, minced', '1 can (14 oz) crushed tomatoes', '1 tablespoon tomato paste', '1 teaspoon dried oregano', '1 teaspoon dried basil', 'Salt and pepper to taste', 'Grated Parmesan cheese for serving', 'Fresh basil leaves for garnish (optional)'] cooking_time=30 servings=4


## Tool Calling

### Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [25]:
from agents import Agent, function_tool

@function_tool
def get_weather(city : str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city : str) -> str:
    print(f"Getting temperature for {city}")
    return "12 degrees"

agent = Agent(
    name= "Weather Agent",
    instructions= "You are the local weather agent. You are given a city and you need to tell the weather and temperature.",
    tools= [get_weather, get_temperature]
)

result = await Runner.run(agent, "Manchester?")
result.final_output

Getting weather for Manchester
Getting temperature for Manchester


'In Manchester, the weather is sunny with a temperature of 12 degrees Celsius.'

In [30]:
from agents import WebSearchTool

news_agent = Agent(
    name = "News Agent",
    instructions= "You are a news reporter. Your job is to find recent news articles on the internet aboust the UK and list them.",
    tools= [WebSearchTool()]
)

result = await Runner.run(news_agent, "Find news")
result.final_output

'Here are some recent news articles related to the United Kingdom:\n\n**Media and Privacy**\n\n- **UK News Coverage at Risk of \'Fracturing Irreparably\' Within 10 Years, Peers Warn**: A House of Lords committee has warned that UK news coverage could fracture irreparably within the next decade due to a growing divide between quality paid journalism and the increasing reliance on questionable online sources. The report highlights the rise of "news deserts" and significant advertising revenue losses in local news, suggesting tax breaks for local media hiring reporters and journalist training schemes to avert a "two-tier" media environment. ([ft.com](https://www.ft.com/content/b062ce44-9fbb-4d5c-a8ec-a58ff1bd8654?utm_source=openai))\n\n- **Murdoch\'s UK Tabloids Apologize to Prince Harry and Admit Intruding on the Late Princess Diana**: Rupert Murdoch\'s UK tabloid newspapers have issued an unprecedented apology to Prince Harry for years of intrusion into his private life and agreed to pa

In [36]:
from agents import WebSearchTool, Runner, agent
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

news_agent = Agent(
    name = "News Agent",
    instructions= "You are a news agent that can search the web for the latest news on a given topic. " +  
    "Compile the information you find into a concise summary. No markdown, just plain text.",
    tools= [WebSearchTool()]
)

# while True:
#     query = input("Enter your news query (or 'quit' to exit): ")
#     if query.lower() == 'quit':
#         break

#     result = Runner.run_sync(news_agent, input = query)
#     print("\nResult:")
#     print(result.final_output)
#     print("\n" + "-"*50 + "\n")

async def main():
    while True:
        query = input("Enter your news query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break
        result = await Runner.run(news_agent, input=query)
        print("\nResult:")
        print(result.final_output)
        print("\n" + "-"*50 + "\n")

# Only needed in Jupyter or async environments
await main()

Enter your news query (or 'quit' to exit):  Oldham



Result:
Here is a summary of recent news and developments in Oldham:

**Public Safety Incidents**

- **Chemical and Gas Leak**: On April 23, 2025, a chemical and gas leak from a lorry on West End Street prompted the evacuation of nearby businesses and buildings. Five fire engines and an incident command unit responded to the scene. Residents were advised to keep windows and doors closed during the incident. ([bbc.com](https://www.bbc.com/news/articles/crkx8rd6l72o?utm_source=openai))

- **Murder Investigation**: On July 26, 2025, a 50-year-old man was found dead in a property on Eldon Street. A 64-year-old man has been arrested on suspicion of murder and remains in custody. The cause of death is yet to be determined. ([bbc.com](https://www.bbc.com/news/articles/cr4zk1r6w5do?utm_source=openai))

**Political Developments**

- **Conservative Leader Resignation**: In January 2025, Graham Sheldon resigned as leader of Oldham Council's Conservative group following a heated council meeting t

Enter your news query (or 'quit' to exit):  quit
